## import modules

In [34]:
import pandas as pd

## pre-process census dataset

In [35]:

# for population data
cols=['State', 'Level', 'Name', 'TRU', 'No_HH', 'TOT_P', 'TOT_M', 'TOT_F']
census=pd.read_excel('datasets/census.xlsx',engine='openpyxl',usecols=cols)
censusIndia=census.iloc[0,:]
census=census.loc[census.Level=='STATE']
census=census.loc[census.TRU=='Total']
census=census.append(censusIndia,ignore_index=True)
census.sort_values(by=['State'],axis=0,inplace=True)
census.reset_index(drop=True,inplace=True)

## repeat of Q1 func 
- with slight modification 
- I have used Q1 function but here I am using ratios rather than percent as ratios are asked in the question

**`NOTE:`** I have not sorted output file by first col; since this question asks for particular sorting. I have outputed files according to that.

In [36]:
#===================
# # a function that just takes state code that does
# manipulates and finds appropriate sum
# finds total pop fro that state from census data
# then returns a nice ratio dictionary
#===================

def ratioFinder(stateCode):
    # 5 is specific to TOT_P
    statePop=census.iloc[stateCode,5] 
    # modify state code
    stateCode=str(stateCode).zfill(2)
    # read df from file
    df=pd.read_excel(f'datasets/C-17/{stateCode}.xlsx',skiprows=6,header=None,engine='openpyxl')
    df.fillna(value=0,inplace=True)
    # find total no 
    # note: these col numbers are specific to df #
    total3=sum(df.iloc[:,14].tolist())
    total2=sum(df.iloc[:,9].tolist())-total3 # need to remove total3 as itself is of 2+ language(that also includes 3+ lang persons) speakers
    total1=sum(df.iloc[:,4].tolist())-total2-total3 # as previously I was double substracting 3+ persons I was getting negative number for that
    # doing -toatal3 because total2 has been modified above with toatl3 being substracted
    ratio_1=total1/statePop # not rounded
    ratio_2=total2/statePop # not rounded
    ratio_3=total3/statePop # not rounded
    #store ratios in a dict
    stateRatios={
        'state-code':stateCode,
        'ratio-one':ratio_1,
        'ratio-two':ratio_2,
        'ratio-three':ratio_3,
    }
    #print('Processed state -- ',stateCode)
    return stateRatios


In [37]:
%%time
ratioList=[ratioFinder(i) for i in range(0,36)]
ratioDF=pd.DataFrame(ratioList)

CPU times: user 19.8 s, sys: 32 ms, total: 19.8 s
Wall time: 19.8 s


### `NOTE:`
- I have included ratios and percent of population in columns of output for information about worst and best state ratios
- also when rounding ratios it gives different(and wrong) states in answer since ratios are quite close for many states; So i have not rounded them for accuracy purposes  

## 2-to-1-ratio part
### sort and store
- sort values as per ratio[top three] and store them into CSV files

In [38]:
ratioDF21=ratioDF.drop(['ratio-three'],axis=1)
ratioDF21['2-to-1-ratio']=ratioDF21['ratio-two']/ratioDF21['ratio-one']
ratioDF21.sort_values('2-to-1-ratio',inplace=True,axis=0,ascending=False)
#print(ratioDF21)
bestStates=ratioDF21.iloc[:3,:]
worstStates=ratioDF21.iloc[-3:,:]
worstStates.sort_values('2-to-1-ratio',axis=0,ascending=True,inplace=True)
bestWorstDF=pd.concat([bestStates,worstStates],axis=0)
#as que asks for specific ordering
#print(bestWorstDF)
bestWorstDF.to_csv('outputs/2-to-1-ratio.csv',index=False)

/tmp/ipykernel_81045/2995019099.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  worstStates.sort_values('2-to-1-ratio',axis=0,ascending=True,inplace=True)


In [39]:
# for 2-to-1-ratio
bestWorstDF

,state-code,ratio-one,ratio-two,2-to-1-ratio
35,35,0.323600,0.485355,1.499862
30,30,0.227894,0.263895,1.157976
12,12,0.359740,0.337779,0.938953
8,08,0.890979,0.094334,0.105877
9,09,0.885525,0.101535,0.114661
22,22,0.867520,0.115938,0.133643


## 3-to-2-ratio part
### sort and store
- sort values as per ratio[top three] and store them into CSV files

In [40]:
ratioDF32=ratioDF.drop(['ratio-one'],axis=1)
ratioDF32['3-to-2-ratio']=ratioDF32['ratio-three']/ratioDF32['ratio-two']
ratioDF32.sort_values('3-to-2-ratio',inplace=True,axis=0,ascending=False)
#print(ratioDF32)
bestStates=ratioDF32.iloc[:3,:]
worstStates=ratioDF32.iloc[-3:,:]
worstStates.sort_values('3-to-2-ratio',axis=0,ascending=True,inplace=True)
bestWorstDF=pd.concat([bestStates,worstStates],axis=0)
#as que asks for specific ordering
#print(bestWorstDF)
bestWorstDF.to_csv('outputs/3-to-2-ratio.csv',index=False)

/tmp/ipykernel_81045/2545202312.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  worstStates.sort_values('3-to-2-ratio',axis=0,ascending=True,inplace=True)


In [41]:
# for 3-to-2-ratio
bestWorstDF

,state-code,ratio-two,ratio-three,3-to-2-ratio
30,30,0.263895,0.508211,1.925807
3,03,0.187624,0.282227,1.504216
4,04,0.244390,0.305063,1.248266
10,10,0.144854,0.016163,0.111582
5,05,0.155650,0.018622,0.119637
9,09,0.101535,0.012940,0.127448


## observations
- notice that state `30` comes in best states in both ratios; meaning it has more population speaking more than one language. It was one of the states which had more 3+ language speaking pop than only one or  exactly two language speaking pop in Q1 
- also notice that state `9` comes in worst states in both ratios